In [1]:
##http://puzzles.bostonpython.com/puddle.html
import os,sys,random,time

In [3]:
def landscape(land, **kwargs):
    
    _h,_v,_b,_r,_w = "_","|", " ", "\n", "~"
    
    ymax, xmax = max(land), len(land)
    _b = _b*2
    vec = (_b * (xmax+1)) + _r
    pvec = len(vec)
    mat = vec * ymax
    
    for x,y in enumerate(land):
        p = (ymax - y) * pvec + (x*2)
        mat = mat[:p] + _h + mat[p+1:]
    
    landdif = []
    for i in range(1,xmax):
        y1, y2 =  land[i-1], land[i]
        landdif.append( range(min(y1,y2),max(y1,y2)) )
    
    for x,v in enumerate(landdif):
        if len(v) < 1: continue
        for y in v:
            p = (ymax - y) * pvec + (x*2) + 1
            mat = mat[:p] + _v + mat[p+1:]
    
    water = kwargs.get('water',[])
    if len(water) > 0:
        for w in water:
            x,y = w[0],w[1]
            p = (1 + ymax - y) * pvec + (x*2)
            mat = mat[:p] + _w + mat[p+1:]
            
    if kwargs.get('legend',False):
        mat = mat + _r + "|".join([str(i) for i in land])
    
    return mat

In [4]:
def fill(land, **kwargs):
    
    '''potential: [ [y0,[x0,x1,...]], [y1,[x0,x1,...]], ...]
        land = [3,1,3] => potential: [[],[ [1,[2]] [2,[2]] ],[]] '''

    ymax,ymin = 0,1000000
    potential, lockedin = [], []
    
    for x,y in enumerate(land):
        
        #ClosedRight: Move Current Potential(up to y) into Lockedin
        if y > ymin:
            if len(potential) > 0:
                temp = [ q for q in potential if q[0] < y]
                if temp is not None: 
                    lockedin.extend(temp)
                    for t in temp:
                        potential.remove(t)
        
        #Open Left-Peak: Extend potential from left peak
        if y < ymax:

            map(lambda p: p[1].append(x), potential)

            for i in range(y,ymax):
                if i not in map(lambda q: q[0], potential):
                    potential.append([i,[x]])
        
        #only y that doesnt catch is y = ymax
        
        #New Globals: ymax tracks highest left peak, ymin tracks yprevious
        if y > ymax: ymax,ymin = y,y
        if y < ymin: ymin = y
        
        if kwargs.get('show_algo',False):
            print "x: ", x, " y: ", y, " ymin: ", ymin, " ymax: ", ymax, " potential: ", potential, " lockedin: ", lockedin

    return lockedin

In [5]:
def fill2xy(fill):
    ret = []
    for layer in fill:
        for x in layer[1]:
            ret.append( (x, layer[0] + 1)  )
    return ret

In [6]:
s = [1,4,1,2,1,3]
out = fill(s)
print out

print landscape(s,legend=True)

print '----------------------------------'

out2 = fill2xy(out)
print out2
print landscape(s,legend=True,water = out2)

[[1, [2]], [2, [2, 3, 4]], [1, [4]]]
  _           
 | |      _   
 | |  _  |    
_| |_| |_|    

1|4|1|2|1|3
----------------------------------
[(2, 2), (2, 3), (3, 3), (4, 3), (4, 2)]
  _           
 | |      _   
 | |~ ~ ~|    
_| |~| |~|    

1|4|1|2|1|3


In [7]:
def showpuddle(s,**kwargs):
    
    out = fill(s, show_algo = kwargs.get('show_algo',False) )
    
    if kwargs.get('show_land', True): print landscape(s,legend=True)
        
    if kwargs.get('show_water', True):
        print '------------------------------------------------'
        print landscape(s,legend=True,water = fill2xy(out))
    
    if kwargs.get('print_volume',False): print "Volume: ", str(len(fill2xy(out)))

In [10]:
land = [2,5,1,2,3,4,7,7,6]
showpuddle(land, print_volume=True)

            _ _     
           |   |_   
  _        |        
 | |      _|        
 | |    _|          
_| |  _|            
   |_|              

2|5|1|2|3|4|7|7|6
------------------------------------------------
            _ _     
           |   |_   
  _        |        
 | |~ ~ ~ ~|        
 | |~ ~ ~|          
_| |~ ~|            
   |~|              

2|5|1|2|3|4|7|7|6
Volume:  10


In [8]:
land = [1,7,5,3,4,2,3,3,6,3,1]
showpuddle(land, print_volume=True)

  _                     
 | |            _       
 | |_          | |      
 |   |  _      | |      
 |   |_| |  _ _| |_     
 |       |_|       |    
_|                 |_   

1|7|5|3|4|2|3|3|6|3|1
------------------------------------------------
  _                     
 | |            _       
 | |~ ~ ~ ~ ~ ~| |      
 |   |~ ~ ~ ~ ~| |      
 |   |~| |~ ~ ~| |_     
 |       |~|       |    
_|                 |_   

1|7|5|3|4|2|3|3|6|3|1
Volume:  16


In [9]:
N = 10
trials = 10
maxY = 10

for t in range(trials):
    land = [random.randint(1, maxY) for i in range(N)]
    showpuddle(land,print_volume=True)
    

    _           _ _   
   | |         |      
   | |_        |      
_  |   |       |      
 | |   |       |      
 | |   |_      |      
 |_|     |  _ _|      
         | |          
         |_|          
                      

7|4|10|8|5|2|4|4|10|10
------------------------------------------------
    _           _ _   
   | |~ ~ ~ ~ ~|      
   | |~ ~ ~ ~ ~|      
_  |   |~ ~ ~ ~|      
 |~|   |~ ~ ~ ~|      
 |~|   |~ ~ ~ ~|      
 |~|     |~ ~ ~|      
         |~|          
         |~|          
                      

7|4|10|8|5|2|4|4|10|10
Volume:  30
  _ _                 
 |   |        _       
 |   |      _| |      
_|   |     |   |      
     |     |   |_     
     |     |     |    
     |  _  |     |    
     | | | |     |    
     |_| | |     |    
         |_|     |_   

7|10|10|2|4|1|8|9|6|1
------------------------------------------------
  _ _                 
 |   |        _       
 |   |~ ~ ~ ~| |      
_|   |~ ~ ~|   |      
     |~ ~ ~|   |_     
     |~ ~ ~|  